# Testing Model Predictions

Versions

- 01
    - Testing for HuggingFace Spaces

## Imports

In [1]:
from importlib.metadata import version
import pandas as pd
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
from torch.nn import Module # For type hinting
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import time
import argparse
from tqdm import tqdm


## Data Pipeline

### Creating Dataset Class

In [ ]:
class OilDataset(Dataset):
    """Dataset class For the OIL_DATASET"""
    def __init__(self, csv_file="../Data/DataSplits/test.csv"):
        try:
            self.data = pd.read_csv(csv_file)   # Assign a pandas data frame
            
        except FileNotFoundError:
            raise FileNotFoundError(f"File not found: {csv_file}")

        # Define feature and label columns
        self.label_column = "Close"
        # Remove the label column from the feature columns
        self.feature_columns = self.data.columns.drop([self.label_column])
        

    def __getitem__(self, index):
        features = self.data.loc[index, self.feature_columns].values    # Extract the features for the given index
        
        label = self.data.loc[index, self.label_column] # Extract the label for the given index
        return (
            torch.tensor(features, dtype=torch.float),
            torch.tensor(label, dtype=torch.float)
        )

    def __len__(self):
        return len(self.data)

### Data Pipeline Function

In [ ]:
def data_pipeline(root_data_dir: str= "../Data", data_file_path: str="OIL_DATASET.csv", data_splits_dir: str="DataSplits", scaler_dir = "Scalers", batch_size: int=64, num_workers=0, pin_memory: bool=False, drop_last: bool=True) -> tuple[Dataset, Dataset, Dataset, DataLoader, DataLoader, DataLoader, MinMaxScaler, MinMaxScaler]:
    """This function prepares the train, test, and validation datasets.
    Args:
        root_data_dir (str): The root of the Data Directory
        data_file_path (str): The name of the original dataset (with .csv file extension).
        data_splits_dir (str): Path to the train, test, and validation datasets.
        scaler_dir (str): Path to the feature and label scalers.
        batch_size (int): The dataloader's batch_size.
        num_workers (int): The dataloader's number of workers.
        pin_memory (bool): The dataloader's pin memory option.
        drop_last (bool): The dataloader's drop_last option.

    Returns: 
        train_dataset (Dataset): Dataset Class for the training dataset.
        test_dataset (Dataset): Dataset Class for the test dataset.
        validation_dataset (Dataset): Dataset Class for the validation dataset.
        train_dataloader (DataLoader): The train dataloader.
        test_dataloader (DataLoader): The test dataloader.
        validation_dataloader (DataLoader): The validation dataloader.
        feature_scaler (MinMaxScaler): The scaler used to scale the features of the model input.
        label_scaler (MinMaxScaler): The scaler used to scale the labels of the model input.
        """
    if not root_data_dir or not data_file_path or not data_splits_dir:  # Check for empty strings at the beginning
        raise ValueError("File and directory paths cannot be empty strings.")
    DATA_ROOT = Path(root_data_dir)
    # OIL_PATH_ORIGINAL = DATA_ROOT / "OIL_Dataset_1984-2025.csv"     # Set the data source path

    DATA_CLEAN_PATH = DATA_ROOT / data_file_path # Set the path to the complete dataset

    if DATA_CLEAN_PATH.exists():
        print(f"CSV file detected, reading from '{DATA_ROOT}'")
        df = pd.read_csv(DATA_CLEAN_PATH)
    else:
        print(f"Downloading CSV file from HuggingFace")
        os.makedirs(DATA_ROOT, exist_ok=True)       # Create the Data Root Directory
        df = pd.read_csv("hf://datasets/MaxPrestige/CRUDE_OIL_PRICES/Data/OIL_DATASET.csv")  # Download and read the data into a pandas dataframe
        df.to_csv(DATA_CLEAN_PATH, index=False)     # Save the file, omitting saving the index

    DATA_SPLITS_DIR = DATA_ROOT / data_splits_dir
    SCALER_DIR = DATA_ROOT / scaler_dir

    TRAIN_DATA_PATH = DATA_SPLITS_DIR / "train.csv"
    TEST_DATA_PATH = DATA_SPLITS_DIR / "test.csv"
    VALIDATION_DATA_PATH = DATA_SPLITS_DIR / "val.csv"

    FEATURE_SCALER_PATH = SCALER_DIR / "feature-scaler.joblib"
    LABEL_SCALER_PATH = SCALER_DIR / "label-scaler.joblib"
    
    label_col = "Close"
    extra_dropped_cols = 'Date'

    if os.path.exists(TRAIN_DATA_PATH) and os.path.exists(TEST_DATA_PATH) and os.path.exists(VALIDATION_DATA_PATH) :
        print(f"Train, Test, and Validation csv datasets detected in '{DATA_SPLITS_DIR}.' Skipping generation and loading scaler(s)")
        try:
            feature_scaler = joblib.load(FEATURE_SCALER_PATH)
            label_scaler = joblib.load(LABEL_SCALER_PATH)
        except Exception as e:
            raise RuntimeError(f"An unexpected error occurred when loading scalers: {e}")
    else:
        print(f"Datasets not found in '{DATA_SPLITS_DIR}' or incomplete. Generating datasets...")
        # os.makedirs(MODEL_ROOT, exist_ok=True)
        os.makedirs(DATA_SPLITS_DIR, exist_ok=True)     # Create the Data Splits Parent Directory
        os.makedirs(SCALER_DIR, exist_ok=True)     # Create the Scaler Parent Directory

        feature_scaler = MinMaxScaler()
        label_scaler = MinMaxScaler()
        # Split the Dataframe into separate features and labels DataFrames
        df_features = df.drop(columns=[label_col, extra_dropped_cols], inplace=False)
        df_labels = df[[label_col]]     # Instead of returning a pandas Series using "[]", return a dataframe using the "[[]]" to get a shape with (-1,1)

        # Split into smaller DataFrames for the Train, Test, and Validation splits
        X_train, X_inter, Y_train, Y_inter = train_test_split(df_features, df_labels, test_size=0.1, random_state=42)
        X_validation, X_test, Y_validation, Y_test = train_test_split(X_inter, Y_inter, test_size=0.5, random_state=42)

        feature_scaler.fit(X_train)
        label_scaler.fit(Y_train)

        # Save the fitted scaler object
        try:
            joblib.dump(feature_scaler, FEATURE_SCALER_PATH)
            print(f"Feature scaler stored in: ({FEATURE_SCALER_PATH})")
            joblib.dump(label_scaler, LABEL_SCALER_PATH)
            print(f"Label scaler stored in: ({LABEL_SCALER_PATH})")
        except Exception as e:
            raise RuntimeError(f"An unexpected error occurred when saving  Scalers: {e}")

        # Scale the rest of the data; returns numpy arrays
        X_train_scaled = feature_scaler.transform(X_train)
        Y_train_scaled = label_scaler.transform(Y_train)
        X_validation_scaled = feature_scaler.transform(X_validation)
        Y_validation_scaled = label_scaler.transform(Y_validation)
        X_test_scaled = feature_scaler.transform(X_test)
        Y_test_scaled = label_scaler.transform(Y_test)

        print(f"Train Features Scaled Shape: {X_train_scaled.shape}")
        print(f"Train Labels Scaled Shape: {Y_test_scaled.shape}")
        print(f"validation Features Scaled Shape: {X_validation_scaled.shape}")
        print(f"validation Labels: {Y_validation_scaled.shape}")
        print(f"test Features Scaled Shape: {X_test_scaled.shape}")
        print(f"test Labels Scaled Shape: {Y_test_scaled.shape}")
        # Define the column names of the features and label
        features_names = df_features.columns
        label_name = df_labels.columns
        # Create dataframes using the scaled data
        X_train_df = pd.DataFrame(X_train_scaled, columns=features_names)
        X_test_df = pd.DataFrame(X_test_scaled, columns=features_names)
        X_validation_df = pd.DataFrame(X_validation_scaled, columns=features_names)
        Y_train_df = pd.DataFrame(Y_train_scaled, columns=label_name)
        Y_test_df = pd.DataFrame(Y_test_scaled, columns=label_name)
        Y_validation_df = pd.DataFrame(Y_validation_scaled, columns=label_name)

        # Concatenate the features and labels back into a single DataFrame for each set
        train_data_frame = pd.concat([X_train_df, Y_train_df.reset_index(drop=True)], axis=1)
        test_data_frame = pd.concat([X_test_df, Y_test_df.reset_index(drop=True)], axis=1)
        validation_data_frame = pd.concat([X_validation_df, Y_validation_df.reset_index(drop=True)], axis=1)

        # Saving the split data to csv files
        train_data_frame.to_csv(TRAIN_DATA_PATH, index=False)
        test_data_frame.to_csv(TEST_DATA_PATH, index=False)
        validation_data_frame.to_csv(VALIDATION_DATA_PATH, index=False)
    # Creating Datasets from the stored datasets
    print(f"Initializing Datasets")
    train_dataset = OilDataset(TRAIN_DATA_PATH)
    test_dataset = OilDataset(TEST_DATA_PATH)
    val_dataset = OilDataset(VALIDATION_DATA_PATH)
    
    print(f"Creating DataLoaders with batch_size ({batch_size}), num_workers ({num_workers}), pin_memory ({pin_memory}). Training dataset drop_last: ({drop_last})")
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=drop_last, shuffle=True)
    validation_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=drop_last, shuffle=False)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=drop_last, shuffle=False)

    print(f"Training DataLoader has ({len(train_dataloader)}) batches, Test DataLoader has ({len(test_dataloader)}) batches, Validation DataLoader has ({len(validation_dataloader)}) batches")

    return (train_dataset, test_dataset, val_dataset, train_dataloader, test_dataloader, validation_dataloader, feature_scaler, label_scaler)

In [4]:
try:
    data_pipeline(root_data_dir="../Data", data_file_path="OIL_DATASET.csv", data_splits_dir="DataSplits")
except Exception as e:
    raise RuntimeError(f"An unexpected error occurred when running the data pipeline function:{e}")

CSV file detected, reading from '..\Data'
Train, Test, and Validation csv datasets detected in '..\Data\DataSplits.' Skipping generation and loading scaler(s)
Initializing Datasets
Creating DataLoaders with batch_size (64), num_workers (0), pin_memory (False). Training dataset drop_last: (True)
Training DataLoader has (89) batches, Test DataLoader has (4) batches, Validation DataLoader has (4) batches


## Agent Architecture

### Module Layer

In [17]:
class ModuleLayer(torch.nn.Module):
    """Class for the individual layer blocks."""
    def __init__(self, intermediate_dim=32, dropout_rate=0.1):
        super().__init__()
        self.mod_linear = torch.nn.Linear(intermediate_dim, intermediate_dim)
        self.mod_norm = torch.nn.LayerNorm(normalized_shape=intermediate_dim)
        self.mod_relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=dropout_rate)

    def forward(self, x):
        """Forward pass of the layer block."""
        residual = x
        x = self.mod_linear(x)
        x = self.mod_norm(x)
        x = self.mod_relu(x)
        x = self.dropout(x)
        x += residual
        return x

### Oil Agent

In [ ]:
class Agent(torch.nn.Module):
    """Class for Agent Structure using multiple Layer Blocks."""
    def __init__(self, cfg):
        super().__init__()
        self.linear = torch.nn.Linear(in_features=cfg["in_dim"], out_features=cfg["intermediate_dim"])
        self.layers = torch.nn.Sequential(*[ModuleLayer(intermediate_dim=cfg["intermediate_dim"], dropout_rate=cfg["dropout_rate"]) for _ in range(cfg["num_blocks"])])
        self.out = torch.nn.Linear(in_features=cfg["intermediate_dim"], out_features=cfg["out_dim"])

    def forward(self, x):
        x = self.linear(x)
        x = self.layers(x)
        x = self.out(x)
        return x
    
    def get_prediction(self, features):
        """Get the deterministic prediction on a single observation or a batch of observations.
        
        Parameters:
            features (torch.tensor): the agent's input features. Expected shape is either `(num_features,)` for a single observation
            or `(batch_size, num_features)` for a batch of observations.
        
        Returns:
            action (int or torch.tensor): 
                - If `features` is a single features (i.e., `features.dim() == 1`), returns a scalar `int` representing the chosen action. 

                - If `features` is a batch of features (i.e., `features.dim() > 1`),
                returns a `torch.Tensor` of `int`s, where each element is the
                chosen action for the corresponding observation in the batch"""
        # Ensure single samples have a batch dimension
        if features.dim() == 1:
            features = features.unsqueeze(0) # Add a batch dimension if it's a single batch of features
        with torch.no_grad():
            if not isinstance(features, torch.Tensor):  # Check if features is not already a tensor
                features = torch.tensor(features, dtype=torch.float32)
            prediction = self.forward(features)  # Run a forward pass through the model
        if features.size(0) == 1:    # This method checks if there is only 1 element in a 1-D tensor
            return prediction.item() # Returns a Python scalar for a single observation
        else:
            return prediction # Returns a tensor of predictions

## Helper Functions

### Log Iteration Functions

In [7]:
def log_iteration(batch_idx: int, total_batches: int, loss_value: float):
    """Logs the loss of the current batch."""
    print(f"Epoch batch [{batch_idx}/{total_batches}] | Loss: {loss_value:.7f}")

In [8]:
def log_epoch_iteration(epoch: int, avg_epoch_loss: float):
    """Log Current Metrics accumulated in the current epoch iteration.
    Args:
        epoch (int): the current iteration
        avg_epoch_loss (float): The average loss of the current epoch
    Returns:
        N/A
        """
    if avg_epoch_loss:
        print(f"=====================  [EPOCH ({epoch}) LOGGING]  =====================")
        print("| AVERAGES of THIS EPOCH:")
        print(f"| ACCUMULATED LOSS: {avg_epoch_loss:.7f}")
        print(f"===========================================================")
    
    else:
        print("No Data collected for this epoch to log")

### Evaluate Model Function

In [9]:
def evaluate_model(model: Module, dataloader: DataLoader, current_epoch: int = None, max_epochs: int=None, device: str = 'cpu') -> float:
    """
    Evaluates the model on a given dataset and returns the average loss.
    Args:
        model (Module): The Model.
        dataloader (DataLoader): The dataloader to calculate average loss with.
        current_epoch (int): The current epoch [optional].
        max_epochs (int): The maximum number of epochs [optional].
        device (str): The device that the calculations will take place on.
    Returns:
        avg_loss (float): The calculated average loss.
    """
    model.eval()
    total_loss = 0.0
    # loss_fn = torch.nn.MELoss(reduction='sum') # Use reduction='sum' instead of 'mean' for total loss
    loss_fn = torch.nn.L1Loss(reduction='sum')
    if len(dataloader.dataset) == 0:
        print("Warning: Evaluation dataset is empty. Skipping evaluation.")
        return float('nan')
    
    with torch.no_grad():
        for batch_inputs, batch_labels in dataloader:
            batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.unsqueeze(dim=-1).to(device)
            outputs = model(batch_inputs)
            loss = loss_fn(outputs, batch_labels)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader.dataset)     # Calculate the average loss on the dataset

    if current_epoch and max_epochs:   # If the function was called in the training loop
        print(f"===================  [Epoch ({current_epoch}/{max_epochs})]  ===================")
        print(f"Entire Validation Dataset Average Loss: {avg_loss:.4f}")
        print(f"====================================================")

    else:   # If the function was called outside of the training loop
        print(f"===============================================")
        print(f"Entire Dataset Average Loss: {avg_loss:.4f} ")
        print(f"=====================================================")
            
    return avg_loss

### Training Function

In [ ]:
def train_model(model_config: dict, train_dataloader: DataLoader, validation_dataloader: DataLoader, model: Agent = None, epochs=32, learning_rate=0.0003, max_grad_norm=0.5, log_iterations=10, eval_iterations=10, device="cpu") -> Agent:
    """The Model Training function.

    Args:
        model_config (dict): The base configurations for building the policies.
        train_dataloader (DataLoader): The dataloader for the training loop.
        validation_dataloader (DataLoader): The dataloader for the validation loop.
        model (Agent): The model to be trained.
        epochs (int): The number of times the outer loop is performed.
        learning_rate (float): The hyperparameter that affects how much the model's parameters learn on each update iteration.
        max_grad_norm (float): Used to promote numerical stability and prevent exploding gradients.
        log_iterations (int): Used to log information about the state of the Agent.
        eval_iterations (int): Used to run an evaluation of the Agent.
        device (str): The device that the model will be trained on.

    Returns: 
        agent (Module): The Trained Model in evaluation mode.
    """
    print(f"Training Model on {device} with {epochs} main epochs, {learning_rate} learning rate, max_grad_norm={max_grad_norm}.")
    print(f"Logging every {log_iterations} epoch iterations, evaluating every {eval_iterations} epoch iterations.")

    agent = (model if model is not None else Agent(model_config)).to(device) # Create agent if nothing was passed, otherwise, create the agent. Send agent to device.

    optimizer = torch.optim.AdamW(params=agent.parameters(), lr=learning_rate, weight_decay=0.01)
    loss_fn = torch.nn.L1Loss(reduction='mean')      # Define the Loss function


    history = {'train_loss': [], 'val_loss': []}

    train_dataloader_length = len(train_dataloader)
    agent.train()   # Set agent to training mode
    for epoch in tqdm(range(epochs), desc=f">>>>>>>>>>>>>>>>>>>>>\nMain Epoch (Outer Loop)", leave=True):

        epoch_loss_total = 0.0
        for batch_idx, (inputs, labels) in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs} - Training", leave=False)):           # Get a mini-batch of training examples from the dataloader
            # optimizer.zero_grad(set_to_none=True)       # Clear the gradients built up; Setting to None to improve performance
            optimizer.zero_grad()       # Clear the gradients built up; Setting to None to improve performance

            inputs, labels = inputs.to(device), labels.unsqueeze(dim=-1).to(device)   # Move the inputs and labels to the device

            agent_outputs = agent(inputs)       # Pass the inputs to the model and get the outputs.

            loss = loss_fn(agent_outputs, labels)      # Calculate the mini-batch loss
            epoch_loss_total += loss.item()
            
            loss.backward()         # Calculate the loss with respect to the model parameters
            torch.nn.utils.clip_grad_norm_(parameters=agent.parameters(), max_norm=max_grad_norm)   # Prevent the gradients from affecting the model parameters too much and reduce the risk of exploding gradients

            optimizer.step()      # Update the model's parameters using the learning rate

            # LOGGING LOSS OF CURRENT ITERATION
            if (batch_idx + 1) % log_iterations == 0:
                log_iteration(batch_idx=(batch_idx + 1), total_batches=train_dataloader_length, loss_value=loss.item())

        # CALCULATE AND STORE THE AVERAGE EPOCH LOSS
        epoch_avg_loss = epoch_loss_total / train_dataloader_length
        history["train_loss"].append(epoch_avg_loss)

        # LOG THE AVERAGE LOSS OF THE EPOCH
        # log_epoch_iteration(epoch=epoch, avg_epoch_loss=epoch_avg_loss)

        # EVALUATE THE MODEL
        if (epoch + 1) % eval_iterations == 0:
            val_loss = evaluate_model(model=agent, dataloader=validation_dataloader, current_epoch=(epoch + 1), max_epochs=epochs, device=device)
            history["val_loss"].append(val_loss)
            agent.train()   # Set agent to training mode
        
    return agent.eval(), history

## Test Trained Policy

In [10]:
cfg = {
    "in_dim": 22,    # Number of Features as input
    "intermediate_dim": 128,    
    "out_dim": 1,   
    "num_blocks": 12,   # Number of reapeating Layer Blocks
    "dropout_rate": 0.1     # Rate for dropout layer
}

In [90]:
trained_policy = Agent(cfg)

In [91]:
SAVE_LOCATION = "./models/Agent_02.pt"

In [92]:
params = torch.load(f=SAVE_LOCATION, weights_only=True)

In [93]:
trained_policy.load_state_dict(params)

<All keys matched successfully>

In [94]:
trained_policy.eval()

Agent(
  (linear): Linear(in_features=22, out_features=128, bias=True)
  (layers): Sequential(
    (0): ModuleLayer(
      (mod_linear): Linear(in_features=128, out_features=128, bias=True)
      (mod_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (mod_relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): ModuleLayer(
      (mod_linear): Linear(in_features=128, out_features=128, bias=True)
      (mod_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (mod_relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (2): ModuleLayer(
      (mod_linear): Linear(in_features=128, out_features=128, bias=True)
      (mod_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (mod_relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (3): ModuleLayer(
      (mod_linear): Linear(in_features=128, out_features=128, bias=True)
      (mod_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
   

In [66]:
# --- Data Preparation Pipeline --- 
try:
    (train_dataset, test_dataset, validation_dataset, train_dataloader, test_dataloader, validation_dataloader, feature_scaler, label_scaler) = data_pipeline(root_data_dir = "../Data", data_file_path = "OIL_DATASET.csv", batch_size=64)
except ValueError as e:
    print(f"Caught an error: {e}")

# --- Testing Trained Model --- 
print("\nTESTING THE TRAINED POLICY:")
test_loss = evaluate_model(model=trained_policy, dataloader=test_dataloader, current_epoch=None, max_epochs=None, device='cpu')

CSV file detected, reading from '..\Data'
Train, Test, and Validation csv datasets detected in '..\Data\DataSplits.' Skipping generation and loading scaler(s)
Initializing Datasets
Creating DataLoaders with batch_size (64), num_workers (0), pin_memory (False). Training dataset drop_last: (True)
Training DataLoader has (89) batches, Test DataLoader has (4) batches, Validation DataLoader has (4) batches

TESTING THE TRAINED POLICY:
Entire Dataset Average Loss: 0.0030 


In [85]:
def sample_model(model, dataset, label_scaler, feature_scaler=None, samples=5) -> None:
    """Function to sample a model on a given dataset and provide a detailed ouput of the predictions, actual values, and loss for each sample.
    Args:
        model (Agent): The Model.
        dataset (Dataset): The dataset to sample from
        samples (int): The number of samples to take from the dataset. If 0 or less, defaults to 5. If more than dataset size, adjusts to dataset size."""
    loss_fn = torch.nn.L1Loss(reduction='sum')  # Declare the los function to be the Mean Absolute Error loss

    if samples <= 0:
        print("Number of samples must be greater than zero. Setting samples to 5")
        samples = 5

    if samples > len(dataset):
        print(f"Requested number of samples ({samples}) exceeds dataset size ({len(dataset)}). Adjusting to dataset size.")
        samples = len(dataset)

    total_loss = 0.0
    with torch.no_grad():
        for index in range(samples):
            (features, label) = dataset[index]
            # print(features[:5])   # Print the first 5 features for inspection
            pred = model(features)  # Get the models prediction baesd on the input features
            loss = loss_fn(pred, label.unsqueeze(dim=-1))   # Calculate the loss, adding an extra dimension to the label to match the prediction shape
            total_loss += loss.item()

            # Un-normalize prediction and label using the inverse function and get the scalar value from the numpy arrays
            pred_unscaled = label_scaler.inverse_transform(pred.cpu().numpy().reshape(-1, 1))[0, 0]

            label_unscaled = label_scaler.inverse_transform(label.cpu().numpy().reshape(-1, 1))[0, 0]

            # Un-normalize features if scaler is provided
            if feature_scaler is not None:
                features_unscaled = feature_scaler.inverse_transform(features.cpu().numpy().reshape(1, -1))[0]
                print(f"Un-normalized features: {features_unscaled}")
            else:
                print(f"Normalized features: {features.numpy()}")

            print(f"Model predicts: {pred.item():.7f} | Actual: {label:.7f} | Loss: {loss.item():.7f}")
            print(f"Un-normalized prediction: {pred_unscaled:.4f} | Un-normalized actual: {label_unscaled:.4f}\n")
    avg_loss = total_loss / samples
    print(f"Average loss over the samples: {avg_loss:.7f}")

In [86]:
sample_model(trained_policy, test_dataset, label_scaler, feature_scaler, samples=2)

Un-normalized features: [ 122.6       124.07      122.579994  112.02      101.1       102.03999
 2521.        444.99997    18.       1009.       3788.       4774.
 8562.         73.         73.       2448.        444.99997  1009.
   18.       3788.0002   4701.       8489.      ]
Model predicts: 0.8267158 | Actual: 0.8185274 | Loss: 0.0081884
Un-normalized prediction: 123.8554 | Un-normalized actual: 122.8000

Un-normalized features: [  60.81      61.88      59.4       58.53      53.11      53.63
 3792.9998   323.        65.       708.      1501.0001  5443.
 6944.0005   501.      2755.      3292.       303.00003  708.
 -201.      1452.      2737.      4189.     ]
Model predicts: 0.3338750 | Actual: 0.3451005 | Loss: 0.0112254
Un-normalized prediction: 60.3332 | Un-normalized actual: 61.7800

Average loss over the samples: 0.0097069


In [63]:
def get_prediction(model, features):
    """Get a prediction from the model for given features.
    Args:
        model (Agent): The Model.
        features (torch.tensor): The input features for prediction.
    Returns:
        prediction (float): The predicted value."""
    model.eval()
    with torch.no_grad():
        if not isinstance(features, torch.Tensor):
            features = torch.tensor(features, dtype=torch.float)
        prediction = model(features.unsqueeze(0))  # Add batch dimension
    return prediction.item()    

In [98]:
sample_input = torch.tensor([[0.703135888501742,0.7017001545595055,0.7101257959310453,0.8316066884769173,0.7677131583462,0.7775022143489813,0.43426550948248155,0.5921052631578947,0.014423076923076924,0.3508036338225017,0.5976363636363636,0.2643835616438355,0.5127923976608186,0.11075949367088608,0.014813224559896953,0.46559139784946235,0.5921052631578947,0.3508036338225017,0.4444444444444444,0.5994570135746606,0.7065111758989311,0.699368840659949],[0.46627951993805655,0.4687017001545596,0.474141947507377,0.49382042893493266,0.42108423400240175,0.4279893711248892,0.7965284474445516,0.3665413533834586,0.1394230769230769,0.15094339622641506,0.527090909090909,0.5118721461187214,0.5411184210526316,0.6787974683544303,0.41155002146844144,0.7408602150537634,0.30263157894736836,0.15094339622641506,0.3170731707317073,0.5236199095022624,0.4586977648202138,0.5116819842763813]], dtype=torch.float)

In [99]:
trained_policy.get_prediction(sample_input)

tensor([[0.7077],
        [0.4703]])

In [88]:
print(f"Model pred: {get_prediction(trained_policy, sample_input)}")

Model pred: 0.7076724171638489
